In [ ]:
import numpy as np
import random
import tensorflow as tf
import sklearn.datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
DIM = 512
CRITIC_ITERS = 1
BATCH_SIZE = 256
L1_lambda = 1
ITERS = 100000

In [ ]:
def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.get_variable('u', [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)

    return w_norm

def snlinear(inputs, input_dim, output_dim, activation, name):
    with tf.variable_scope(name):
        weights = tf.get_variable(
            'kernel',
            [input_dim, output_dim],
            initializer=tf.random_normal_initializer(0., output_dim ** -0.5)
        )
        biases = tf.get_variable('bias', [output_dim])
        outputs = tf.matmul(inputs, spectral_norm(weights)) + biases
        if activation:
            outputs = activation(outputs)

        return outputs

def Generator(inputs, name):
    with tf.variable_scope('Generator/' + name, reuse=tf.AUTO_REUSE):
        outputs = tf.layers.dense(inputs, DIM, activation=tf.nn.relu, name='Generator.1')
        outputs = tf.layers.dense(outputs, DIM, activation=tf.nn.relu, name='Generator.2')
        outputs = tf.layers.dense(outputs, DIM, activation=tf.nn.relu, name='Generator.3')
        outputs = tf.layers.dense(outputs, 2, activation=None, name='Generator.4')
        return outputs

def Discriminator(inputs, name):
    with tf.variable_scope('Discriminator/' + name, reuse=tf.AUTO_REUSE):
        outputs = snlinear(inputs, 2, DIM, activation=tf.nn.relu, name='Discriminator.1')
        outputs = snlinear(outputs, DIM, DIM, activation=tf.nn.relu, name='Discriminator.2')
        outputs = snlinear(outputs, DIM, DIM, activation=tf.nn.relu, name='Discriminator.3')
        outputs = snlinear(outputs, DIM, 1, activation=None, name='Discriminator.4')
        return outputs

In [ ]:
real_X = tf.placeholder(tf.float32, shape=[None, 2], name='real_X')
real_Y = tf.placeholder(tf.float32, shape=[None, 2], name='real_Y')

fake_X = Generator(real_Y, 'Y2X')
fake_Y = Generator(real_X, 'X2Y')

disc_real_X = Discriminator(real_X, 'X')
disc_fake_X = Discriminator(fake_X, 'X')

disc_real_Y = Discriminator(real_Y, 'Y')
disc_fake_Y = Discriminator(fake_Y, 'Y')

recons_X = Generator(fake_Y, 'Y2X')
recons_Y = Generator(fake_X, 'X2Y')

disc_loss_X = tf.reduce_mean(disc_fake_X) - tf.reduce_mean(disc_real_X)
gen_loss_Y2X = -tf.reduce_mean(disc_fake_X)

disc_loss_Y = tf.reduce_mean(disc_fake_Y) - tf.reduce_mean(disc_real_Y)
gen_loss_X2Y = -tf.reduce_mean(disc_fake_Y)

disc_loss = disc_loss_X + disc_loss_Y
gen_loss = gen_loss_X2Y + \
            gen_loss_Y2X + \
            L1_lambda * tf.reduce_mean(tf.abs(real_X - recons_X)) + \
            L1_lambda * tf.reduce_mean(tf.abs(real_Y - recons_Y))

In [ ]:
disc_params = tf.trainable_variables('Discriminator')
gen_params = tf.trainable_variables('Generator')

disc_train_op = tf.train.AdamOptimizer(
    learning_rate=3e-4,
    beta1=0.0,
    beta2=0.9
).minimize(disc_loss, var_list=disc_params)

gen_train_op = tf.train.AdamOptimizer(
    learning_rate=1e-4,
    beta1=0.0,
    beta2=0.9
).minimize(gen_loss, var_list=gen_params)

In [ ]:
def create_real_X():
    swiss_roll_data = sklearn.datasets.make_swiss_roll(
        n_samples=BATCH_SIZE,
        noise=0.3
    )[0]
    swiss_roll_data = swiss_roll_data.astype('float32')[:, [0, 2]]
    swiss_roll_data /= 7.5
    return swiss_roll_data

def create_real_Y():
    s_curve_data = sklearn.datasets.make_s_curve(
        n_samples=BATCH_SIZE,
        noise=0.05
    )[0]
    s_curve_data = s_curve_data.astype('float32')[:, [0, 2]]
    return s_curve_data

def generate_image():
    real_X_data = create_real_X()
    fake_X_data = sess.run(fake_X, {real_Y: create_real_Y()})

    real_Y_data = create_real_Y()
    fake_Y_data = sess.run(fake_Y, {real_X: create_real_X()})

    plt.figure()

    plt.subplot(121)
    plt.scatter(real_X_data[:, 0], real_X_data[:, 1], c='orange', marker='+')
    plt.scatter(fake_X_data[:, 0], fake_X_data[:, 1], c='green', marker='+')
    
    plt.subplot(122) Y_data[:, 0], real_Y_data[:, 1], c='orange', marker='+')
    plt.scatter(fake_Y_data[:, 0], fake_Y_data[:, 1], c='green', marker='+')

    plt.show()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
%%time

for i in range(ITERS+1):
    for _ in range(CRITIC_ITERS):
        fd = {
            real_X: create_real_X(),
            real_Y: create_real_Y(),
        }
        _disc_loss, _ = sess.run([disc_loss, disc_train_op], fd)

    sess.run(gen_train_op, fd)

    if i % 10000 == 0:
        print(i, _disc_loss)
        generate_image()
